# Prediction of Traffic Accident Risk
Team 7: 어서오십쇼HUMAN  
Editor: 허윤행, 김용현
## 1. Preprocess

In [ ]:
# 사전 준비
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings

plt.style.use('seaborn')
sns.set(font_scale=2.5)

warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
import matplotlib
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
matplotlib.rcParams['axes.unicode_minus'] = False   


In [ ]:
sample = pd.read_csv('../input/TAAS_data.csv')

print("sample.shape:", sample.shape)

# 초기 샘플 데이터
sample

# 2. 결측값 제거 

In [ ]:
# 제거해주세용 - 바로 진행합니다~
sample.replace('불명',np.nan,inplace = True)
sample

In [ ]:
sample.isnull().sum()

In [ ]:
sample = sample.dropna()
sample

In [ ]:
#기타/불명 -> 불명으로 변경 후 삭제 데이터 7만 얼마에서 69000대로 줄어든 거 확인!!!!

# 3. label 변경

In [ ]:
## 범주형 데이터 -> 수치형 데이터
# 사고내용
Target = {1: "사망", 2 : "중상", 3 : "경상"}

for label in Target:
    sample['사고내용'] = np.where(sample['사고내용'] == Target[label], label, sample['사고내용'])


# 사고 시 특이사항
Detail = {0: "특이사항 없음", 1: "신호등 꺼짐상태", 2: "신호등 이상", 3: "보행자 신호", 4: "인도 위 사고", 5: "노면상태 결함", 6: "미끄러움", 7: "질퍽한 도로"}

for label in Detail:
    sample['사고 시 특이사항'] = np.where(sample['사고 시 특이사항'] == Detail[label], label, sample['사고 시 특이사항'])

# 기상 상태
Weather = {1 : "맑음", 2 : "비", 3: "눈", 4: "바람", 5: "폭풍우", 6: "눈보라", 7: "안개", 8: "기타", 9: "불명"}

for label in Weather:
    sample['기상상태'] = np.where(sample['기상상태'] == Weather[label], label, sample['기상상태'])

# 가해자성별
Sex = {1: "남성", 2: "여성"}

for label in Sex:
    sample['가해자성별'] = np.where(sample['가해자성별'] == Sex[label], label, sample['가해자성별'])

# 가해차종
CarType = {1: '자전거',2: '50cc', 3: '125cc', 4:'500cc', 5: '500cc 이상',8: '택시',9: '자가용',10: '승합차',11: '버스', 16: '기타', 17: '농기계', 18: '트럭', 19: '3.5t급 미만', 20: '7.5t급 미만', 21: '7.5t급 이상', 22: '스쿠터',23: '전기차'}

for label in CarType:
    sample['가해차종'] = np.where(sample['가해차종'] == CarType[label], label, sample['가해차종'])
    
# 노면상태
RoadState = {1: '건조',2: '젖음',3: '눈',4: '얼음',5: '3cm이상 물에 잠김',6: '미끄러움',7: '질퍽한 도로'}

for label in RoadState:
    sample['노면상태'] = np.where(sample['노면상태'] == RoadState[label], label, sample['노면상태'])

# 빛 세기
Light = {1:'밝음',4: '조명 있음', 5: '어두움',6: '조명 없음',7: '기타/불명'}

for label in Light:
    sample['빛 세기'] = np.where(sample['빛 세기'] == Light[label], label, sample['빛 세기'])

# 사고 시 통제 방법

Control = {0: '20m내에 교차로 없음', 1: '교통 정리 경찰',2: '신호등',3: '정지 신호',4: '일방 통행'}
for label in Control:
    sample['사고 시 통제 방법'] = np.where(sample['사고 시 통제 방법'] == Control[label], label, sample['사고 시 통제 방법'])
JunLocation = { 0 : '20m내에 교차로 없음',1 : '교차로 접근로',2 : '비어있는 교차로',3 : '회전교차로 출구',4 : '회전 교차로 입구',5 : '주 도로 출구',6 : '주 도로 입구',7 : '경사로 진입로',8 : '위험로'}
for label in JunLocation:
    sample['사고 위치'] = np.where(sample['사고 위치'] == JunLocation[label],label,sample['사고 위치'])
sample    

In [ ]:
## 범주형 데이터 -> 원핫 인코딩
# To Do List:
# 요일, 보행자이동, 사고 위치
sample

In [ ]:
sample = pd.get_dummies(sample,columns=['요일'],prefix=['요일'])
sample = pd.get_dummies(sample,columns=['보행자이동'],prefix=['보행자이동'])
sample

In [ ]:
sample.dtypes

In [ ]:
#object 형태에서 int형으로 통일하기 위한 작업
#우선 float형태가 있기 때문에 float로 갔다가 int로 변경
sample = sample.astype('float')
sample = sample.astype('int')
sample.dtypes

In [ ]:
sample

# 3. logarithm 함수 써서 정규화(나이)

In [ ]:
# 코드 복붙 예정

# 4. 상관관계 분석 

In [ ]:
# ~

# 5. 시각화

In [ ]:
# ~

# 6. 데이터 저장

In [ ]:
# sample.to_csv('../input/final.csv', encoding='utf-8-sig',index_label=False)